In [1]:
import os

import numpy as np

import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt

from StyleFrame import StyleFrame, Styler, utils

In [2]:
writer = StyleFrame.ExcelWriter('out.xlsx')

In [3]:
weights = DataFrame()
file = 'scores.xlsx'

raw_product = pd.read_excel(file, sheet_name='Product', skiprows=2, usecols=3)
raw_product['page'] = 'product'
raw_company = pd.read_excel(file, sheet_name='Company', skiprows=2, usecols=3)
raw_company['page'] = 'company'

raw = raw_product.append(raw_company)
raw = raw.reset_index(drop=True)
raw.columns = ['requirement', 'score', 'notes', 'weight', 'page']
del raw['score']
del raw['notes']


# starting at the last category
# extract the rows for the category to the end of the df
# remove the rows
header_indexes = raw.index[raw['requirement'] == 'Requirement'].tolist()
header_indexes = list(reversed(header_indexes))

for hi in header_indexes:
    category = raw.iloc[hi-1][0]
    sub_weights = DataFrame(raw.iloc[hi+1:])
    sub_weights['category'] = category
    sub_weights = sub_weights[pd.notnull(sub_weights['requirement'])]

    weights = weights.append(sub_weights)
    raw = raw.iloc[0:hi-1]

    
weights = weights.reset_index(drop=True)
weights

,requirement,weight,page,category
0,D&B Rating,5,company,Corporate Viability
1,Years in business,1,company,Corporate Viability
2,Hold time less than 5 minutes,5,company,Technical Support
3,Resolution on first call,5,company,Technical Support
4,Polite,2,company,Technical Support
5,Pleasant apppearance,1,product,Packaging
6,Easy to open,1,product,Packaging
7,Recyclable,2,product,Packaging
8,Must be able to hold 20 oz,5,product,Technical Specs
9,Flexible,2,product,Technical Specs


In [4]:
scores = DataFrame()
files = [f for f in os.listdir('.') if f.endswith('-scores.xlsx')]

for file in files:
    raw = pd.read_excel(file, sheet_name='Product', usecols=1, header=None)
    scorer, product = raw.iloc[0][1], raw.iloc[1][1]

    raw_product = pd.read_excel(file, sheet_name='Product', skiprows=2, usecols=2)
    raw_product['page'] = 'product'
    raw_company = pd.read_excel(file, sheet_name='Company', skiprows=2, usecols=2)
    raw_company['page'] = 'company'


    raw = raw_product.append(raw_company)
    raw = raw.reset_index(drop=True)
    raw.columns = ['requirement', 'score', 'notes', 'page']

    # starting at the last category
    # extract the rows for the category to the end of the df
    # remove the rows
    header_indexes = raw.index[raw['requirement'] == 'Requirement'].tolist()
    header_indexes = list(reversed(header_indexes))

    for hi in header_indexes:
        category = raw.iloc[hi-1][0]
        sub_scores = DataFrame(raw.iloc[hi+1:])
        sub_scores['category'] = category
        sub_scores['scorer'] = scorer
        sub_scores['product'] = product
        sub_scores = sub_scores[pd.notnull(sub_scores['requirement'])]

        scores = scores.append(sub_scores)
        raw = raw.iloc[0:hi-1]

    
scores = scores.reset_index(drop=True)
scores.head()


,requirement,score,notes,page,category,scorer,product
0,D&B Rating,NaN,john acme 3,company,Corporate Viability,Billy Kidd,Acme Widget
1,Years in business,NaN,john acme 5,company,Corporate Viability,Billy Kidd,Acme Widget
2,Hold time less than 5 minutes,NaN,john acme 5,company,Technical Support,Billy Kidd,Acme Widget
3,Resolution on first call,NaN,john acme 5,company,Technical Support,Billy Kidd,Acme Widget
4,Polite,NaN,john acme 2,company,Technical Support,Billy Kidd,Acme Widget


In [5]:
cleansed = scores.dropna(axis=0, subset=['score'])
cleansed.loc[:,'score'] = pd.to_numeric(cleansed.loc[:,'score'])

sf = StyleFrame(cleansed)
sf.apply_column_style(cols_to_style=['notes'], width=20,styler_obj=Styler(wrap_text=True))
sf.to_excel(writer, sheet_name='Scores', best_fit=['requirement', 'score', 'notes', 'page', 'category', 'scorer', 'product'])

cleansed

/Users/ctarrington/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:621: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,requirement,score,notes,page,category,scorer,product
9,Flexible,5,billy acme 5,product,Technical Specs,Billy Kidd,Acme Widget
11,D&B Rating,4,jane acme 4,company,Corporate Viability,Jane Doe,Acme Widget
12,Years in business,5,jane acme 5,company,Corporate Viability,Jane Doe,Acme Widget
13,Hold time less than 5 minutes,3,jane acme 3,company,Technical Support,Jane Doe,Acme Widget
14,Resolution on first call,5,jane acme 5,company,Technical Support,Jane Doe,Acme Widget
15,Polite,5,jane acme 5,company,Technical Support,Jane Doe,Acme Widget
16,Pleasant apppearance,4,jane acme 4,product,Packaging,Jane Doe,Acme Widget
17,Easy to open,1,jane acme 1,product,Packaging,Jane Doe,Acme Widget
19,Must be able to hold 20 oz,5,jane acme 5,product,Technical Specs,Jane Doe,Acme Widget
20,Flexible,2,jane acme 2,product,Technical Specs,Jane Doe,Acme Widget


In [6]:
minimums = cleansed.sort_values('score').groupby(['product','page', 'category', 'requirement'], as_index=False).first()
maximums = cleansed.sort_values('score').groupby(['product','page', 'category', 'requirement'], as_index=False).last()
means = cleansed.groupby(['product','page', 'category', 'requirement'], as_index=False)['score'].mean()
counts = cleansed.groupby(['product','page', 'category', 'requirement'], as_index=False).size().reset_index(name='responses')

minimums = minimums.rename({'score': 'minimum score', 'notes': 'minimum notes', 'scorer': 'minimum scorer'}, axis='columns')
maximums = maximums.rename({'score': 'maximum score','notes': 'maximum notes', 'scorer': 'maximum scorer'}, axis='columns')
means = means.rename({'score': 'mean score'}, axis='columns')

aggregate = pd.merge(minimums, maximums, on=['product','page', 'category', 'requirement'])
aggregate = pd.merge(aggregate, means, on=['product','page', 'category', 'requirement'])
aggregate = pd.merge(aggregate, counts, on=['product','page', 'category', 'requirement'])
aggregate = pd.merge(aggregate, weights, on=['page', 'category', 'requirement'])
aggregate['mean contribution'] = aggregate['mean score'] * aggregate['weight']

aggregate

,product,page,category,requirement,minimum score,minimum notes,minimum scorer,maximum score,maximum notes,maximum scorer,mean score,responses,weight,mean contribution
0,Acme Widget,company,Corporate Viability,D&B Rating,3,john acme 3,John Doe,4,jane acme 4,Jane Doe,3.5,2,5,17.5
1,Romco Widget,company,Corporate Viability,D&B Rating,2,john romco 2,John Doe,2,john romco 2,John Doe,2.0,1,5,10
2,Acme Widget,company,Corporate Viability,Years in business,5,john acme 5,John Doe,5,jane acme 5,Jane Doe,5.0,2,1,5
3,Romco Widget,company,Corporate Viability,Years in business,2,john romco 2,John Doe,3,Jane romco 3,Jane Doe,2.5,2,1,2.5
4,Acme Widget,company,Technical Support,Hold time less than 5 minutes,3,jane acme 3,Jane Doe,5,john acme 5,John Doe,4.0,2,5,20
5,Romco Widget,company,Technical Support,Hold time less than 5 minutes,2,Jane romco 2,Jane Doe,3,john romco 3,John Doe,2.5,2,5,12.5
6,Acme Widget,company,Technical Support,Polite,2,john acme 2,John Doe,5,jane acme 5,Jane Doe,3.5,2,2,7
7,Romco Widget,company,Technical Support,Polite,2,Jane romco 2,Jane Doe,3,john romco 3,John Doe,2.5,2,2,5
8,Acme Widget,company,Technical Support,Resolution on first call,5,john acme 5,John Doe,5,jane acme 5,Jane Doe,5.0,2,5,25
9,Romco Widget,company,Technical Support,Resolution on first call,3,john romco 3,John Doe,4,Jane romco 4,Jane Doe,3.5,2,5,17.5


In [7]:
summary_product = aggregate['mean contribution'].groupby(aggregate['product']).sum().to_frame()
summary_product.columns = ['total weighted score']
summary_product['count'] = aggregate['mean contribution'].groupby(aggregate['product']).count()
summary_product

,total weighted score,count
product,,
Acme Widget,115.0,10
Romco Widget,84.0,11


In [8]:
summary_scorer = cleansed['score'].groupby(cleansed['scorer']).mean().to_frame()
summary_scorer.columns = ['average score']
summary_scorer['count'] = cleansed['score'].groupby(cleansed['scorer']).count()

summary_scorer

,average score,count
scorer,,
Billy Kidd,5.0,1
Jane Doe,3.2,20
John Doe,2.6,20


In [9]:
cleansed['min score'] = cleansed.groupby(['product','page','category', 'requirement'])['score'].transform('min')
cleansed['max score'] = cleansed.groupby(['product','page','category', 'requirement'])['score'].transform('max')
cleansed['mean score'] = cleansed.groupby(['product','page','category', 'requirement'])['score'].transform('mean')
cleansed['responses'] = cleansed.groupby(['product','page','category', 'requirement'])['score'].transform('count')
cleansed = cleansed.sort_values(by=['product','page', 'category','requirement', 'score'], ascending=[True,True,True,True,False])
cleansed = cleansed[['product','page', 'category','requirement','scorer','score','notes','min score','mean score','max score','responses']]
cleansed

/Users/ctarrington/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ctarrington/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ctarrington/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

,product,page,category,requirement,scorer,score,notes,min score,mean score,max score,responses
11,Acme Widget,company,Corporate Viability,D&B Rating,Jane Doe,4,jane acme 4,3,3.5,4,2
33,Acme Widget,company,Corporate Viability,D&B Rating,John Doe,3,john acme 3,3,3.5,4,2
12,Acme Widget,company,Corporate Viability,Years in business,Jane Doe,5,jane acme 5,5,5.0,5,2
34,Acme Widget,company,Corporate Viability,Years in business,John Doe,5,john acme 5,5,5.0,5,2
35,Acme Widget,company,Technical Support,Hold time less than 5 minutes,John Doe,5,john acme 5,3,4.0,5,2
13,Acme Widget,company,Technical Support,Hold time less than 5 minutes,Jane Doe,3,jane acme 3,3,4.0,5,2
15,Acme Widget,company,Technical Support,Polite,Jane Doe,5,jane acme 5,2,3.5,5,2
37,Acme Widget,company,Technical Support,Polite,John Doe,2,john acme 2,2,3.5,5,2
14,Acme Widget,company,Technical Support,Resolution on first call,Jane Doe,5,jane acme 5,5,5.0,5,2
36,Acme Widget,company,Technical Support,Resolution on first call,John Doe,5,john acme 5,5,5.0,5,2


In [10]:
writer.save()